In [ ]:
import pandas as pd
import datetime
import numpy as np
from forex_python.bitcoin import BtcConverter
from datetime import datetime
import urllib.parse
import urllib.request
import requests
import json

API_KEY = 'e796791dfc9a03772797a8a23f972a0c98157b699d45add99035dead71f40fa6'

In [ ]:
add_df = pd.read_csv('../data/address.csv', skiprows=[1])
address_to_name = pd.Series(add_df.Name.values, index = add_df.Adress).to_dict()
token_name = 'OMG'
agg_type = 'receiver'
file_name = "../data/" + token_name + "_data.csv"

In [ ]:
df_on_chain = pd.read_csv(file_name, index_col=[0])
df_on_chain["date"] = pd.to_datetime(df_on_chain['time']).dt.strftime("%Y-%m-%d")

In [ ]:
df_on_chain = df_on_chain[df_on_chain['token_sender'].isin(address_to_name.keys()) | df_on_chain['token_receiver'].isin(address_to_name.keys())]
df_on_chain.reset_index(inplace=True, drop=True)

In [ ]:
df_on_chain['sender'] = df_on_chain.apply(lambda row: address_to_name[row.token_sender] if row.token_sender in address_to_name else row.token_sender, axis = 1).copy()
df_on_chain['receiver'] = df_on_chain.apply(lambda row: address_to_name[row.token_receiver] if row.token_receiver in address_to_name else row.token_receiver, axis = 1).copy()

In [ ]:
df_on_chain['time'] = pd.to_datetime(df_on_chain['time']).dt.floor('H').copy()
df_on_chain['time']

In [ ]:
df_on_chain = df_on_chain[df_on_chain[agg_type].isin(address_to_name.values())]

In [ ]:
if agg_type == 'sender':
    summary = df_on_chain.groupby(
         ['date', 'time','sender']
     ).agg(
         sum_amount_from = ('amount','sum'),
         token     = ('token','min'),
    #      sender = ('sender', 'min'),
    #      receiver = ('receiver' , 'min')
     ).reset_index()

    summary['ts'] = summary.apply(lambda row: int(row.time.timestamp()), axis=1)
    summary = summary.pivot_table(columns=['sender'], index=['time','ts','token'], values='sum_amount_from').reset_index()

else:
    summary = df_on_chain.groupby(
         ['date', 'time','receiver']
     ).agg(
         sum_amount_to = ('amount','sum'),
         token     = ('token','min'),
     ).reset_index()

    summary['ts'] = summary.apply(lambda row: int(row.time.timestamp()), axis=1)
    summary = summary.pivot_table(columns=['receiver'], index=['time','ts','token'], values='sum_amount_to').reset_index()

In [ ]:
summary=summary.fillna(0)
summary.head()

In [ ]:
senders, receivers = df_on_chain.sender.unique().tolist(), df_on_chain.receiver.unique().tolist()

exchanges = set(receivers)
if agg_type == 'sender':
    exchanges = set(senders)

exchanges = list(exchanges)

In [ ]:
def get_px_data(symbol, from_ts, to_ts, to="BTC", exchange=None,limit=2000):
    params = {'fsym' : symbol, 'tsym': to, 'limit': limit, 'api_key': API_KEY, 'toTs':to_ts}
    if exchange:
        params['e'] = exchange
    url = "https://min-api.cryptocompare.com/data/v2/histohour?%s" % urllib.parse.urlencode(params)
    data = json.load(urllib.request.urlopen(url))
    if data['Response'] == 'Error':
        return None, None
    dataframe = pd.json_normalize(data['Data']['Data']) 
    while int(from_ts) < data['Data']['TimeFrom']:
        params['toTs']  =  data['Data']['TimeFrom']
        url = "https://min-api.cryptocompare.com/data/v2/histohour?%s" % urllib.parse.urlencode(params)
        data = json.load(urllib.request.urlopen(url))
        temp = pd.json_normalize(data['Data']['Data'])
        dataframe = pd.concat([dataframe, temp])
    
    return data['Data']['TimeFrom'], dataframe

In [ ]:
min_ts, max_ts = summary.ts.min(), summary.ts.max()
min_ts, max_ts
new_df = pd.DataFrame(summary[['ts']].copy())
new_df.head()
e = []
for exchange in exchanges:
    
    time_from, data = get_px_data("BNB", min_ts, max_ts,to = "BTC", exchange=exchange)
    if time_from == None:
        print("Not found for " + str(exchange))
        continue
    print(exchange)
    e.append(exchange)
    col_name = "price_" + exchange
    col_name2 = "volumeto_" + exchange
    col_name3 = "volumefrom_" + exchange
#     new_data = data[["time","close"]].copy()
#     new_data = new_data.rename(columns={'close': col_name,'time':'ts' })
    new_data = data[["time","close","volumeto","volumefrom"]].copy()
    new_data = new_data.rename(columns={'close': col_name,'time':'ts','volumeto':col_name2,'volumefrom':col_name3 })
    new_df = pd.merge(new_df, new_data, how='right', on=['ts'])

new_df['datetime'] = pd.to_datetime(new_df['ts'], unit='s')

In [ ]:
df_on_chain = summary.copy()
df_off_chain = new_df.copy()
e = ['ts']
e.extend([i.replace('price_','') for i in list(df_off_chain.columns) if 'price' in i  ])
df_off_chain = df_off_chain[df_off_chain['ts'].isin(df_on_chain['ts'])]

print(e)
# e = sorted(e)
df_on_chain = df_on_chain[df_on_chain.columns.intersection(e)]
df_on_chain = df_on_chain.reindex(columns=e)

new_on_chain = df_on_chain.copy()
new_off_chain = df_off_chain.copy()
new_on_chain = new_on_chain.set_index('ts')
new_on_chain = new_on_chain.sort_values(by='ts',ascending=True)
new_off_chain = new_off_chain.sort_values(by='ts',ascending=True)


for col in e[1:]:
    new_name = 'price_' + col
    new_name2 = 'volumeto_' + col
    new_name3 = 'volumefrom_' + col
    new_off_chain[new_name] = new_off_chain[new_name].replace(0,0.0000001)
    new_off_chain[new_name2] = new_off_chain[new_name2].replace(0,1)
    new_off_chain[new_name3] = new_off_chain[new_name3].replace(0,1)
    new_off_chain[col] = new_off_chain[new_name].divide(new_off_chain[new_name].shift()) - 1
    new_off_chain[new_name2] = new_off_chain[new_name2].divide(new_off_chain[new_name2].shift()) - 1

    new_off_chain[new_name3] = new_off_chain[new_name3].divide(new_off_chain[new_name3].shift()) - 1

    del new_off_chain[new_name]

new_off_chain = new_off_chain[1:]
new_off_chain = new_off_chain.set_index('ts')
new_off_chain.to_csv('../data/BNB_price_volume_data.csv')
new_off_chain = new_off_chain[new_off_chain.index.isin(new_on_chain.index)]
new_off_chain = new_off_chain.groupby(['ts']).max()


In [ ]:
cols = new_off_chain.columns.tolist()
cols.insert(0, cols.pop(cols.index('datetime')))
new_off_chain = new_off_chain.reindex(columns=cols)

In [ ]:
new_off_chain.head()

In [ ]:
new_off_chain = new_off_chain.drop(new_off_chain[new_off_chain['KuCoin']>0][:1].index)
new_off_chain = new_off_chain.drop(new_off_chain[new_off_chain['HitBTC']>0][:1].index)
# new_off_chain = new_off_chain.drop(new_off_chain[new_off_chain['Coinex']>0][:1].index)
new_off_chain = new_off_chain.drop(new_off_chain[new_off_chain['Bitmax']>0][:1].index)

new_on_chain = new_on_chain[new_on_chain.index.isin(new_off_chain.index)]
new_off_chain.max()

In [ ]:
new_off_chain['volumefrom_Binance'].max()
new_off_chain['volumefrom_Binance'].idxmax(axis=0, skipna=True)
new_off_chain['volumefrom_Binance'].argmax()

new_off_chain.iloc[9765:9768]

new_off_chain[new_off_chain.index == 1557925200].index

idx = new_off_chain.index.get_loc(1557925200)
new_off_chain.iloc[idx - 2 : idx + 2]

In [ ]:
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm

def test_adf(series, title=''):
    dfout={}
    dftest=sm.tsa.adfuller(series.dropna(), autolag='AIC', regression='ct')
    for key,val in dftest[4].items():
        dfout[f'critical value ({key})']=val
    if dftest[1]<=0.05:
        print("Strong evidence against Null Hypothesis")
        print("Reject Null Hypothesis - Data is Stationary")
        print("Data is Stationary", title)
    else:
        print("Strong evidence for  Null Hypothesis")
        print("Accept Null Hypothesis - Data is not Stationary")
        print("Data is NOT Stationary for", title)

In [ ]:
test_adf(new_off_chain['volumeto_Binance'], " Stock Price")

In [ ]:
a = "to_"
if agg_type == 'sender':
    a = "from_"
file2 = token_name + "_ammount_" + a 
save_file = "../data/" + file2 + "price_diff_volume_diff.csv"

new_off_chain.to_csv(save_file,header=True)

file3 = "../data/" + token_name + "_amount_" + a +'data.csv'

new_on_chain.to_csv(file3,header=True)